In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

2023-10-15 18:50:13.727951: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-15 18:50:14.870708: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-15 18:50:16.292107: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-15 18:50:16.292549: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [ ]:
! pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=c69f82161b5e7c253a26d29a3892eb63750ead9538f73e29a21bb8edbc2e04e1
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
!pip install transformers -U -q
import torch
import tensorflow as tf
import spacy
import nltk
import inflect
from num2words import num2words  # Import num2words function from the library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.4 MB/s eta 0:00:00


In [ ]:
! pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.7 MB/s eta 0:00:00


In [ ]:
!pip freeze | grep transformers

transformers==4.34.0


In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

### Loading Mbart Model

In [ ]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")

### Loading Tokenizer

In [ ]:
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")

In [ ]:
def is_noun(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    for token in doc:
        if token.pos_ in ("NOUN", "PROPN"):
            return True
        if token.pos_ == "ADV" and token.text.endswith("ly"):
            return True
    return False

In [ ]:
from inflect import engine
import inflect

def translate(input_text):
    model_inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    # translate from English to Hindi
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"],
        num_beams=5  # Use 5 beams for translation
    )

    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    # Split the input text into words
    words = input_text[0].split()

    # Initialize a list to store nouns
    retain_list = []
    p = engine()
    number_list = {}

    # Check if each word is a noun and add it to the list
    for word in words:
        if is_noun(word):
            retain_list.append(word)
        elif word.isnumeric():
            number_word = p.number_to_words(word)
            number_list[word] = number_word  # Store the translated version

    if len(retain_list) > 0:
        # Check if there are nouns to translate
        translated_sentence = translation.copy()
        translated_sentence = translated_sentence[0]

        # print(translated_sentence)
        # Create a dictionary to store noun translations
        word_translations = {}

        for word in retain_list:
            word = word.replace(',', '')

            # Translate the noun to Hindi
            model_inputs_word = tokenizer(word, return_tensors="pt", padding=True, truncation=True)
            generated_tokens_word = model.generate(
                **model_inputs_word,
                forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"],
                num_return_sequences=4  # Get multiple translations
            )
            translations = tokenizer.batch_decode(generated_tokens_word, skip_special_tokens=True)
            word_translations[word] = translations

            # Replace nouns with translations in the translated_sentence
            for word, translations in word_translations.items():
                for translation in translations:
                    if translation in translated_sentence:
                        translated_sentence = translated_sentence.replace(translation, word)
        # print(number_list)
        for numeric_word, translated_word in number_list.items():
            word_translations = {}
            numeric_word = numeric_word.replace(',', '')

            # Translate the number to Hindi
            model_inputs_word = tokenizer(translated_word, return_tensors="pt", padding=True, truncation=True)
            generated_tokens_word = model.generate(
                **model_inputs_word,
                forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"],
            )
            translations = tokenizer.batch_decode(generated_tokens_word, skip_special_tokens=True)[0]
            word_translations[numeric_word] = translations
            # print(word_translations)
            # Replace translated numbers with original numeric words in the translated_sentence
            for translated_word, translations in word_translations.items():
              # print(translated_word, translations)
              translated_sentence = translated_sentence.replace(translated_word, translations)

        return translated_sentence
    else:
        # Handle the case when there are no nouns to translate
        translated_sentence = translation.copy()
        translated_sentence = translated_sentence[0]
        return translated_sentence


# HardCode Sentence

In [ ]:
translate(['I had about a 30 minute demo just using this new headset'])

'मैं सिर्फ इस नए headset के उपयोग के बारे में एक तीस minute demo था'

# GUI


In [ ]:
from IPython.display import HTML, display
import ipywidgets as widgets
import time

style = """
<style>
    body {
        font-family: Arial, sans-serif;
    }
    .container {
        max-width: 400px;
        margin: 20px auto;
        padding: 20px;
        background-color: #f9f9f9;
        border: 1px solid #ccc;
        border-radius: 10px;
        text-align: center;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
    }
    .heading{
      max-width: 100%;
        margin: 10px auto;
        padding: 20px;
        background-color: #f9f9f9;
        border-radius: 10px;
        text-align: center;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
    }
    .input-box {
        width: 100%;
        padding: 10px;
        border: 1px solid #ccc;
        border-radius: 5px;
        margin-bottom: 10px;
    }
    .translate-button {
        background-color: #007BFF;
        color: white;
        border: none;
        padding: 0px 10px;
        text-align: center;
        text-decoration: none;
        display: inline-block;
        font-size: 14px;
        margin: 2px 2px;
        cursor: pointer;
        border-radius: 5px;
    }
    .progress-bar {
        width: 100%;
        background-color: #ccc;
        border-radius: 5px;
        padding: 3px;
    }
    .progress {
        width: 0;
        height: 20px;
        border-radius: 5px;
        background-color: #007BFF;
    }
</style>
"""

display(HTML(f'<div class="heading"><div><h2>Im Your English To Hinglish Transformer!</h2></div></div>'), display_id='progress')


# translate input
def translate_and_display(input_text):
    # 3-second translation process
    for i in range(1, 101):
        time.sleep(0.03)
        progress = i

        display(HTML(f'<div class="heading"><div><h2>Translating...</h2></div><div class="progress-bar"><div class="progress" style="width: {progress}%"></div></div></div>'), display_id='progress')

    translated_text = translate([input_text])
    display(HTML(f'<div class="container"><p><strong>Translated Text:</strong></p><p>{translated_text}</p></div>'))

# handle button click
def on_button_click(b):
    input_text = text_input.value
    translate_and_display(input_text)

# a text input and a button
text_input = widgets.Text(
    value='',
    placeholder='Enter text in English',
    description='',
    disabled=False,
    layout=widgets.Layout(width="100%")
)

button = widgets.Button(description="Translate")
button.add_class("translate-button")
button.on_click(on_button_click)

# Display the input field and button
display(HTML(style))
display(widgets.HBox([text_input, button]))